# **Problem 1** #

## 1. ##
Proteins labelled with Cy5 -> Cy5 has excitation peak at ~ 649 nm, and emission peak at ~ 667 nm.

DNA labelled with Cy3 -> Cy3 has excitation peak at ~ 554 nm, and emission peak at ~ 566 nm.

<img src="setup.png" alt="setup" style="width:450px;"/> From "Fluorescence Correlation Spectroscopy
An Introduction to its Concepts and Applications", Petra Schwille and Elke Haustein

**Sample:**
A dilute solution (nanomolar range) containing the two species is prepared on a coverslip or within a sealed chamber. The low concentration is crucial for ensuring that only a few molecules reside in the focal volume at any given time.

**Objective:**
The objective lens must have a high numerical aperture (ideally NA > 0.9) in order to focus the laser light to a sufficiently small focal volume to ensure the number of simultaneously detected particles is not too large. Water immersion objectives are often used.

**Lasers:**
